In [1]:
import json
import time
import pickle

import requests
import pandas as pd

url_main = 'https://api.hh.ru/vacancies'
param = "?"
hosts = ["host=hh.ua", "host=hh.ru", "host=career.ru", "host=jobs.tut.by",
        "host=jobs.day.az", "host=hh.uz", "host=hh.kz", "host=headhunter.ge",
       "host=headhunter.kg"]
an = "&"
per_p = "per_page="
page = "page="
url =  'https://api.hh.ru/vacancies?host=hh.ua&per_page=100&page='

In [2]:
r = requests.get('https://api.hh.ru/vacancies?host=hh.ua&per_page=100&page=0')
print(r.status_code)
#print(r.text)
#vacancies = json.loads(r.text)
#print(json.loads(r.text)["items"][0])
#print()
#print(json.loads(r.text)["items"][0]["snippet"])
len(json.loads(r.text)["items"])

200


100

In [3]:


def load_vacancies_ids(url, hosts):
    vacancies = []
    vac_ids = []
    param = "?"
    an = "&"
    per_page = "per_page=100"
    page = "page="
    for host in hosts:
        for i in range(20):
            url_mod = url + param + host
            url_mod = url_mod + an + per_page + an + page + str(i)
            r = requests.get(url_mod)
            vac = json.loads(r.text)
            vacancies = vacancies + vac["items"]
            vac_ids = vac_ids + [i["id"] for i in vac["items"]]
    return vac_ids, vacancies



In [4]:
%%time
vac_ids, vacancies = load_vacancies_ids(url_main, hosts)

Wall time: 1min 41s


In [5]:
len(vac_ids)

18000

In [6]:
ids_set = set(vac_ids)

In [7]:
len(ids_set)

3815

In [13]:
list(ids_set)[100]

'25978699'

In [8]:
with open("headHunter_data/hh_ids.dat", "rb") as inf:
    ids2_list = pickle.load(inf)
    
with open("headHunter_data/hh_vacancies.dat", "rb") as inf:
    vacancies2 = pickle.load(inf)
    
with open("headHunter_data/hh_vacancies_ext.dat", "rb") as inf:
    vacancies_ext2 = pickle.load(inf)

In [9]:
print(len(vacancies_ext2))
print(len(ids2_list))
print(len(ids2_list + list(ids_set)))

ids_all = set(ids2_list + list(ids_set))

print(len(ids_all))

14913
14372
18187
16487


In [14]:
len(ids_set.intersection(set(ids2_list)))

1700

In [16]:
unic_ids = ids_set.difference(ids_set.intersection(set(ids2_list)))
print(len(unic_ids))

2115


In [17]:
vacancies = vacancies2 + vacancies
print(len(vacancies))

90000


In [18]:
ids_all = set(ids2_list + list(unic_ids))
print(len(ids_all))

16487


In [19]:
with open("hh_ids.dat", "wb") as ouf:
    pickle.dump(list(ids_all), ouf)
    
with open("hh_vacancies.dat", "wb") as ouf:
    pickle.dump(vacancies, ouf)

In [20]:

def load_vacancies_extended(url, ids):
    vacancies_ext = []
    count = 0
    for i in ids:
        url_mod = url + "/" + str(i)
        r = requests.get(url_mod)
        vacancy = json.loads(r.text)
        vacancies_ext.append(vacancy)
        count += 1
        
        if count % 500 == 0:
            print(count)
    return vacancies_ext


In [21]:
%%time
vacancies_ext = load_vacancies_extended(url_main, list(unic_ids))

500
1000
1500
2000
Wall time: 8min 53s


In [22]:
print(len(vacancies_ext))
print(len(vacancies_ext2))

2115
14913


In [23]:
vacancies_ext = vacancies_ext2 + vacancies_ext
print(len(vacancies_ext))

17028


In [24]:
with open("hh_vacancies_ext.dat", "wb") as ouf:
    pickle.dump(list(vacancies_ext), ouf)

In [ ]:

with open("hh_ids.json", "w") as ouf:
    json.dump(list(ids_all), ouf, ensure_ascii=False)
    
with open("hh_vacancies.json", "w") as ouf:
    json.dump(vacancies, ouf, ensure_ascii=False)
    
with open("hh_vacancies_ext.json", "w") as ouf:
    json.dump(vacancies_ext, ouf, ensure_ascii=False)
    

In [ ]:
with open("hh_ids.json", "w") as ouf:
    json.dump(list(ids_all), ouf, ensure_ascii=False)

In [ ]:
df = pd.DataFrame(vacancies)

In [ ]:
df.head()

In [18]:
with open("headHunter_data/hh_vacancies.dat", "rb") as inf:
    t = pickle.load(inf)

In [19]:
len(t)

54000

In [20]:
with open("headHunter_data/hh_vacancies_ext.dat", "rb") as inf:
    t = pickle.load(inf)
len(t)    

11112

In [21]:
with open("headHunter_data/hh_ids.dat", "rb") as inf:
    t = pickle.load(inf)
len(t)  

11061

In [25]:
with open("hh_ids.dat", "rb") as inf:
    t = pickle.load(inf)
print(len(t))

with open("hh_vacancies.dat", "rb") as inf:
    t = pickle.load(inf)
print(len(t))

with open("hh_vacancies_ext.dat", "rb") as inf:
    t = pickle.load(inf)
print(len(t))    

16487
90000
17028
